In [1]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf

In [2]:
train = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test = pd.read_csv('data/test_features.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
y = tf.keras.utils.to_categorical(train_labels['label']) 

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, BatchNormalization, Dropout
from keras import optimizers

def deep_lstm():
    model = Sequential()
    model.add(LSTM(32, input_shape = (600,6), return_sequences = True))
    model.add(Dropout(0.2)) 
    model.add(LSTM(64, return_sequences = True))
    model.add(Dropout(0.2)) 
    model.add(LSTM(128, return_sequences = False))
    model.add(Dropout(0.2)) 
    model.add(Dense(128, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    
    #adam = optimizers.adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model

In [8]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [9]:
model1 = deep_lstm()
hist = model1.fit(X,y, epochs=100, batch_size=32, verbose = 1,  validation_split=0.2, callbacks = [es])

Epoch 1/100
79/79 [==============================] - 116s 1s/step - loss: 3.1541 - accuracy: 0.4173 - val_loss: 2.8159 - val_accuracy: 0.4944
Epoch 2/100
79/79 [==============================] - 122s 2s/step - loss: 2.6933 - accuracy: 0.4804 - val_loss: 2.5334 - val_accuracy: 0.4752
Epoch 3/100
79/79 [==============================] - 143s 2s/step - loss: 2.4496 - accuracy: 0.4904 - val_loss: 2.4830 - val_accuracy: 0.4784
Epoch 4/100
79/79 [==============================] - 143s 2s/step - loss: 2.4044 - accuracy: 0.4904 - val_loss: 2.3364 - val_accuracy: 0.5200
Epoch 5/100
79/79 [==============================] - 143s 2s/step - loss: 2.2453 - accuracy: 0.5216 - val_loss: 2.2582 - val_accuracy: 0.5232
Epoch 6/100
79/79 [==============================] - 132s 2s/step - loss: 2.1618 - accuracy: 0.5283 - val_loss: 2.1517 - val_accuracy: 0.5312
Epoch 7/100
79/79 [==============================] - 102s 1s/step - loss: 2.0974 - accuracy: 0.5141 - val_loss: 2.1387 - val_accuracy: 0.5328
Epoch 

In [10]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape
prediction=model1.predict(test_X)
submission.iloc[:,1:]=prediction
submission.to_csv('output/deepLSTM_es_drop_submission.csv', index=False)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()